In [144]:
from openai import OpenAI
import os
import json
from sklearn.metrics.pairwise import cosine_similarity
import requests
from dotenv import load_dotenv
# Load environment variables
load_dotenv()


True

In [19]:
def get_chatbot_response(client,model_name,messages,temperature=0):
    input_messages = []
    for message in messages:
        input_messages.append({"role": message["role"], "content": message["content"]})

    response = client.chat.completions.create(
        model=model_name,
        messages=input_messages,
        temperature=temperature,
        top_p=0.8,
        max_tokens=2000,
    ).choices[0].message.content
    
    return response

In [2]:
client = OpenAI(
    api_key=os.getenv('RUNPOD_TOKEN'),
    base_url=os.getenv('RUNPOD_CHATBOT_URL'),
)
model_name = os.getenv("MODEL_NAME")

## Gte LLM response

In [21]:
messages = [{'role':'user','content':"What's the capital of Italy?"}]
response = get_chatbot_response(client,model_name,messages)
print(response)

The capital of Italy is Rome (Italian: Roma).


In [20]:
response.choices[0].message.content

'The capital of the United States of America is Washington, D.C. (short for District of Columbia).'

## Prompt Engineering

### Structured Output

In [22]:
system_prompt = """
You are a helpful assistant that answer questions about capitals of countries.

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
[
{
    "country": "the country that you will get the capital of" 
    "capital": "the capital of the country stated"
}
]
"""

# Create messages
messages = [{'role':'system','content':system_prompt}]
messages.append({'role':'user','content':"What's the capital of Italy?"})

# Get response

response = get_chatbot_response(client,model_name,messages)
print(response)

[
{
    "country": "Italy",
    "capital": "Rome"
}
]


In [25]:
type(response)

json_response = json.loads(response)
json_response
type(json_response[0]),json_response[0]['capital']


(dict, 'Rome')

## Input Structuring

In [26]:
user_prompt = """
Get me the capitals of the following countries:
```
1. Argentina
2. Brazil
3. Israel
```
"""
messages = [{'role':'system','content':system_prompt}]
messages.append({'role':'user','content':user_prompt})
response = get_chatbot_response(client,model_name,messages)
print(response)


[
    {
        "country": "Argentina",
        "capital": "Buenos Aires"
    },
    {
        "country": "Brazil",
        "capital": "Brasília"
    },
    {
        "country": "Israel",
        "capital": "Jerusalem"
    }
]


In [27]:
json_response = json.loads(response)
json_response

[{'country': 'Argentina', 'capital': 'Buenos Aires'},
 {'country': 'Brazil', 'capital': 'Brasília'},
 {'country': 'Israel', 'capital': 'Jerusalem'}]

## Give the Model time to think (Chain of thought LLM model)
https://arxiv.org/pdf/2205.11916

In [29]:
user_prompt = """
Calculate the result of this equation: 259/2*8654+91072*33-12971

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{
    steps: This is where you solve the equation bit by bit following the BEDMAS order of operations. You need to show your work and calculate each step leading to final result. Feel free to write here in free text. 
    result: The final number resulted from calculating the equation above
}
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

{
    "steps": "First, calculate the division: 259/2 = 129.5. Then, multiply 129.5 by 8654: 129.5 * 8654 = 112,141.5. Next, multiply 91072 by 33: 91072 * 33 = 3,003,336. Now, add 112,141.5 and 3,003,336: 112,141.5 + 3,003,336 = 4,115,477.5. Finally, subtract 12971 from 4,115,477.5: 4,115,477.5 - 12971 = 4,005,806.5.",
    "result": 4005806.5
}


## RAG - Retrival Augmented Generation

##### RAG = helps the model to output info that it dosent have in the memorey inject info in the promt so that user can refer the data and response to it

In [30]:
user_prompt = """
What's new in iphone 16?
tell me about the 3d camera
"""

messages = [{'role': 'user', 'content': user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

I think there may be some confusion here!

There is no such thing as an "iPhone 16". The latest iPhone models available in the market are iPhone 13 series, which includes iPhone 13, iPhone 13 Pro, and iPhone 13 Pro Max.

However, I can tell you about the camera features of the latest iPhone 13 series, including the 3D camera.

The iPhone 13 Pro and Pro Max models feature a triple-camera setup, which includes:

1. Wide-angle camera: 12MP, f/1.8 aperture, 26mm focal length
2. Telephoto camera: 12MP, f/2.0 aperture, 52mm focal length
3. Ultra-wide-angle camera: 12MP, f/2.4 aperture, 13mm focal length

The 3D camera, also known as the "Time-of-Flight" (ToF) camera, is a new feature introduced in the iPhone 13 Pro and Pro Max. It uses a dedicated camera sensor to capture depth information, allowing for advanced features like:

* Depth Control: allows you to adjust the depth of field in Portrait mode photos
* Night mode portraits: enables better low-light performance in Portrait mode
* Impro

In [152]:
iphone_16 = """
The iPhone 16 introduces several exciting updates, making it one of Apple's most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.

Powered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.

Apple also introduced advanced audio features like "Audio Mix," which uses machine learning to separate background sounds from speech, allowing for more refined audio capture during video recording. Battery life has been extended, especially in the iPhone 16 Pro Max, which is claimed to have the longest-lasting battery of any iPhone 
9TO5MAC

APPLEMAGAZINE
.

Additionally, Apple has switched to USB-C for faster charging and data transfer, and the Pro models now support up to 2x faster video encoding. The starting prices remain consistent with previous generations, with the iPhone 16 starting at $799, while the Pro models start at $999
"""

In [153]:
user_prompt = f"""
{iphone_16}

what's new in iphon 16?
"""
messages = [{'role': 'user', 'content': user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

According to the text, the new features and updates in the iPhone 16 include:

1. Larger displays:
	* Base model: 6.1-inch
	* iPhone 16 Plus: 6.7-inch
	* iPhone 16 Pro: 6.3-inch
	* iPhone 16 Pro Max: 6.9-inch
2. Thinner bezels and more durable Ceramic Shield
3. New A18 chip (A18 Pro for Pro models) with:
	* Enhanced neural engine capabilities
	* Faster GPU for gaming and machine learning tasks
4. Upgraded camera systems:
	* Base iPhone 16: Dual-camera setup with 48MP main sensor
	* Pro models: 48MP Ultra Wide and 5x telephoto camera
	* "Camera Control" button for more flexible photography options
5. Advanced audio features:
	* "Audio Mix" for separating background sounds from speech during video recording
6. Extended battery life, especially in the iPhone 16 Pro Max
7. Switch to USB-C for faster charging and data transfer
8. Pro models support up to 2x faster video encoding
9. Starting prices remain consistent with previous generations: $799 for the base iPhone 16 and $999 for the Pro 

## Automatically Extract context data from database

Embanding the proses of changing text from word to an array 

In [155]:
samsung_s23 = """
The Samsung Galaxy S23 brings some incremental but notable upgrades to its predecessor, the Galaxy S22. It features the Snapdragon 8 Gen 2 processor, a powerful chip optimized for the S23 series, delivering enhanced performance, especially for gaming and multitasking. This chip ensures top-tier speed and efficiency across all models, from the base S23 to the larger S23+ and S23 Ultra​
STUFF

TECHRADAR
.

In terms of design, the S23's camera module has been streamlined by removing the raised metal contour around the cameras, creating a cleaner, sleeker look. It also sports the same 6.1-inch 120Hz AMOLED display, protected by tougher Gorilla Glass Victus 2, making it more resistant to scratches and drops​
TECHRADAR
.

The S23 Ultra stands out with its 200MP main camera, offering impressive photo clarity, especially in low-light conditions. The selfie camera across the series has been updated to a 12MP sensor, resulting in sharper selfies. The Ultra model also includes productivity tools such as the S-Pen, which remains an essential feature for note-taking and creative tasks​
STUFF

TECHRADAR
.

Battery life is solid, with the S23 Ultra featuring a 5000mAh battery that lasts comfortably through a day of heavy use. However, charging speeds still lag behind some competitors, with 45W wired charging, which is slower than other brands offering up to 125W charging​
STUFF
.

Overall, the Galaxy S23 series enhances performance, durability, and camera quality, making it a strong contender for users seeking a high-performance flagship.
"""


In [172]:
data = [iphone_16,samsung_s23]

In [157]:
user_prompt = """What's new in iphon 16?"""

In [158]:
emebdding_client = OpenAI(
    api_key=os.getenv('RUNPOD_TOKEN'),
    base_url=os.getenv('RUNPOD_EMBEDDING_URL'),
)

In [173]:
def get_embedding(embedding_client,model_name,text_input):
    output = embedding_client.embeddings.create(input = text_input,model=model_name)
    
    embedings = []
    for embedding_object in output.data:
        embedings.append(embedding_object.embedding)

    return embedings

In [174]:
user_prompt_embedding = get_embedding(emebdding_client, model_name, user_prompt)[0]

In [175]:
data_embeddings = [get_embedding(emebdding_client, model_name, text)[0] for text in data]

In [176]:
data_similarity = cosine_similarity([user_prompt_embedding], data_embeddings)

In [177]:
data_similarity

array([[0.56563478, 0.44444391]])

In [178]:
len(data_embeddings)

2

In [179]:
closest_entry_index = data_similarity.argmax()
closest_entry_index

np.int64(0)

In [168]:
print(data_similarity)


[[0.44444391 0.56563478]]


In [180]:
data[closest_entry_index]

'\nThe iPhone 16 introduces several exciting updates, making it one of Apple\'s most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.\n\nPowered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.\n\nApple also introduced advanced audio features like "Audio Mix," which uses machine l

In [181]:
user_prompt_with_data = f"""
{data[closest_entry_index]}

{user_prompt}
""" 

In [182]:
print(user_prompt_with_data)



The iPhone 16 introduces several exciting updates, making it one of Apple's most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.

Powered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.

Apple also introduced advanced audio features like "Audio Mix," which uses machine learnin

In [183]:
messages = [{'role':'user','content':user_prompt_with_data}]
response = get_chatbot_response(client,model_name,messages)
print(response)

According to the text, the new features and updates in the iPhone 16 include:

1. Larger displays: 6.1-inch for the base model, 6.7-inch for the iPhone 16 Plus, 6.3 inches for the iPhone 16 Pro, and 6.9 inches for the iPhone 16 Pro Max.
2. Thinner bezels and a more durable Ceramic Shield.
3. New A18 chip (A18 Pro for the Pro models) with improved performance, neural engine capabilities, and faster GPU for gaming and machine learning tasks.
4. Upgraded camera systems:
	* Base iPhone 16: dual-camera setup with a 48MP main sensor.
	* Pro models: 48MP Ultra Wide and 5x telephoto camera, with Apple's "Camera Control" button for more flexible photography options.
5. Advanced audio features, including "Audio Mix" for more refined audio capture during video recording.
6. Extended battery life, especially in the iPhone 16 Pro Max, which has the longest-lasting battery of any iPhone.
7. Switch to USB-C for faster charging and data transfer.
8. Pro models support up to 2x faster video encoding.
9